In [23]:
import numpy  as np
import pandas as pd
import random as rnd

In [24]:
distance = np.array([[0.00, 28.02,  17.12, 27.46, 46.07],
            [28.02, 0.00,  34.00, 25.55, 25.55],
            [17.12, 34.00,  0.00, 18.03, 57.38],
            [27.46, 25.55, 18.03,  0.00, 51.11],
            [46.07, 25.55, 57.38, 51.11,  0.00]])

decision_vars = [0,1,2,3,4]
num_individuals = 10
num_generation = 200
crossver_prob = 0.87
mutation_prob = 0.7

In [25]:
def fitness_function(sol):
    #this will be the entry point from vtr
    total_distance = 0
    for i in range(len(sol)-1):
        total_distance += distance[decision_vars.index(sol[i]),decision_vars.index(sol[i+1])]
    return total_distance

In [26]:
def initiate():
    population_bag = []
    for i in range(num_individuals):
        rnd_sol = decision_vars.copy()
        rnd.shuffle(rnd_sol)
        population_bag.append(rnd_sol)
    return np.array(population_bag)

In [27]:
def eval_fit_pop(population_bag):
    res = {}
    fit_val_list = []
    sols = []
    for sol in population_bag:
        fit_val_list.append(fitness_function(sol))
        sols.append(sol)
    res["fit_val"] = fit_val_list
    min_wgt = [np.max(list(res["fit_val"]))-i for i in list(res["fit_val"])] #normalizing it?
    res["fit_wgt"] = [i/sum(min_wgt) for i in min_wgt]
    res["sol"] = np.array(sols)
    return res

In [34]:
def pick(population_bag):
    fit_bag_evals = eval_fit_pop(population_bag)
    t = True
    while t:
        rnIndex = rnd.randint(0, len(population_bag)-1)
        rnPick = fit_bag_evals["fit_wgt"][rnIndex]
        r = rnd.random()
        if r <= rnPick:
            pickedSol = fit_bag_evals["sol"][rnIndex]
            t = False
    return pickedSol

In [35]:
def crossover(solA, solB):
    n = len(solA)
    offspring = [np.nan for i in range(n)]
    num_els = np.ceil(n*(rnd.randint(10,90)/100)) #??
    str_pnt = rnd.randint(0,n-2)
    end_pnt = n if int(str_pnt+num_els) > n else int(str_pnt+num_els)
    blockA = list(solA[str_pnt:end_pnt])
    offspring[str_pnt:end_pnt] = blockA
    for i in range(n):
        if list(blockA).count(solB[i]) == 0:
            for j in range(n):
                if np.isnan(offspring[j]):
                    offspring[j] = solB[i]
                    break
    return offspring

In [36]:
def swap(sol, pos_1, pos_2):
    result = sol.copy()
    el_1 = sol[pos_1]
    el_2 = sol[pos_2]
    result[pos_1] = el_2
    result[pos_2] = el_1
    return result

In [37]:
def mutation(sol):
    n = len(sol)
    pos_1 = rnd.randint(0,n-1)
    pos_2 = rnd.randint(0,n-1)
    result = swap(sol,pos_1,pos_2)
    return result

In [38]:
# ====================Main function===========================
pop_bag = initiate()

for gen in range(num_generation):
    
    pop_bag_fit = eval_fit_pop(pop_bag)

    best_fit = np.min(pop_bag_fit["fit_val"])
    best_fit_index = pop_bag_fit["fit_val"].index(best_fit)
    best_sol = pop_bag_fit["sol"][best_fit_index]

    if gen == 0:
        best_fit_global = best_fit
        best_sol_global = best_sol
    else:
        if best_fit <= best_fit_global:
            best_fit_global = best_fit
            best_sol_global = best_sol
    
    new_pop_bag = []

    for i in range(num_individuals):
        pA = pick(pop_bag)
        pB = pick(pop_bag)
        new_element = pA

        if rnd.random() <= crossver_prob:
            new_element = crossover(pA,pB)

        if rnd.random() <= mutation_prob:
            new_element = mutation(new_element)

        new_pop_bag.append(new_element)

    pop_bag = np.array(new_pop_bag)

    print(f"Best fitness: {best_fit_global}")
    print(f"Best solution: {best_sol_global}")
    #This should be the exiting point from this file

Best fitness: 88.72
Best solution: [3 2 0 1 4]
Best fitness: 86.25
Best solution: [0 2 3 1 4]
Best fitness: 86.25
Best solution: [0 2 3 1 4]
Best fitness: 86.25
Best solution: [0 2 3 1 4]
Best fitness: 86.25
Best solution: [0 2 3 1 4]
Best fitness: 86.25
Best solution: [0 2 3 1 4]
Best fitness: 86.25
Best solution: [0 2 3 1 4]
Best fitness: 86.25
Best solution: [0 2 3 1 4]
Best fitness: 86.25
Best solution: [0 2 3 1 4]
Best fitness: 86.25
Best solution: [4 1 3 2 0]
Best fitness: 86.25
Best solution: [4 1 3 2 0]
Best fitness: 86.25
Best solution: [4 1 3 2 0]
Best fitness: 86.25
Best solution: [0 2 3 1 4]
Best fitness: 86.25
Best solution: [0 2 3 1 4]
Best fitness: 86.25
Best solution: [0 2 3 1 4]
Best fitness: 86.25
Best solution: [0 2 3 1 4]
Best fitness: 86.25
Best solution: [0 2 3 1 4]
Best fitness: 86.25
Best solution: [0 2 3 1 4]
Best fitness: 86.25
Best solution: [0 2 3 1 4]
Best fitness: 86.25
Best solution: [0 2 3 1 4]
Best fitness: 86.25
Best solution: [0 2 3 1 4]
Best fitness:

In [ ]:
$VTR_ROOT/vpr/vpr \
    $VTR_ROOT/vtr_flow/arch/timing/EArch.xml \
    $VTR_ROOT/vtr_flow/benchmarks/blif/tseng.blif \
    --route_chan_width 100 \
    --analysis --disp on